In [1]:
import os

In [2]:
%pwd

'/Users/sanket/Documents/ML Project/Full-Stack Machine Learning Deployment with MLOps and AWS EC2/research'

In [3]:
os.chdir('../')


In [4]:
%pwd

'/Users/sanket/Documents/ML Project/Full-Stack Machine Learning Deployment with MLOps and AWS EC2'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [6]:
from MlOpsProject.constants import *
from MlOpsProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema =  self.schema.Target_Column

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column = schema.name
        )

        return model_trainer_config

In [8]:
import pandas as pd
import os
from MlOpsProject import logger
from sklearn.linear_model import ElasticNet
import joblib

In [9]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):
        try:
            train_data = pd.read_csv(self.config.train_data_path)
            test_data = pd.read_csv(self.config.test_data_path)

            train_x = train_data.drop([self.config.target_column], axis=1)
            test_x = test_data.drop([self.config.target_column], axis=1)
            train_y = train_data[[self.config.target_column]]
            test_y = test_data[[self.config.target_column]]

            # Logging the shapes of train and test data
            logger.info(f"Training data shape: {train_x.shape}, Target shape: {train_y.shape}")
            logger.info(f"Test data shape: {test_x.shape}, Target shape: {test_y.shape}")

            lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
            lr.fit(train_x, train_y)

            joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))
            logger.info(f"Model saved at {os.path.join(self.config.root_dir, self.config.model_name)}")

        except Exception as e:
            logger.exception("Error during model training")
            raise e


In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-10-05 17:13:43,290: INFO: common: YAML file: config/config.yaml loaded successfully.]
[2024-10-05 17:13:43,294: INFO: common: YAML file: params.yaml loaded successfully.]
[2024-10-05 17:13:43,296: INFO: common: YAML file: schema.yaml loaded successfully.]
[2024-10-05 17:13:43,297: INFO: common: created directory at: artifacts]
[2024-10-05 17:13:43,297: INFO: common: created directory at: artifacts/model_trainer]
[2024-10-05 17:13:43,321: INFO: 3075602831: Training data shape: (1199, 11), Target shape: (1199, 1)]
[2024-10-05 17:13:43,322: INFO: 3075602831: Test data shape: (400, 11), Target shape: (400, 1)]
[2024-10-05 17:13:43,336: INFO: 3075602831: Model saved at artifacts/model_trainer/model.joblib]
